<a href="https://colab.research.google.com/github/ufofon2/korea/blob/master/06_rag_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 환경변수 설정


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

### Pinecone 벡터스토어를 이용한 유사도 검색


In [ ]:
from pinecone import Pinecone

# Pinecone 객체를 초기화합니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스 이름을 설정합니다.
index_name = "wiki"

# 인덱스를 초기화합니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 출력합니다.
index.describe_index_stats()

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 객체를 초기화합니다.
# 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)

In [ ]:
from langchain_pinecone import PineconeVectorStore

# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "full_text"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="full_text")

In [ ]:
# 질문을 정의합니다.
question = "벨기에(Belgium)는 어디 있나요?"

# 질문에 대한 유사도 검색을 수행합니다.
# query는 question을 사용해, wiki-ns1 네임스페이스의 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question, k=5, namespace="wiki-ns1")
print(docs)

# 검색된 문서의 메타데이터를 출력합니다.
for doc in docs:
    print(doc.metadata)

### 프롬프트에서 RAG 결과 사용하기


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# ChatOpenAI 객체를 초기화합니다.
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

# 여행할 국가를 설정합니다.
country = "벨기에(Belgium)"

# ChatPromptTemplate 객체를 초기화합니다.
# 시스템 메시지와 사용자 메시지를 포함한 프롬프트 템플릿을 정의합니다.
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 베테랑 여행 가이드로, 고객 최적의 맞춤 여행 일정 수립을 도와줍니다."),
    ("human", """저는 {country} 여행을 계획하고 있습니다.
                 {country}에서 무엇을 할 수 있을지 조언해 주세요.
                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요.
                 context:
                 {context}"""),
])

# 유사도 검색을 위한 쿼리를 정의합니다.
query = f"{country}는 어디 있나요?"

# 벡터 스토어에서 유사도 검색을 수행하여 상위 5개의 결과를 반환합니다.
results = vector_store.similarity_search(query=query, k=5, namespace="wiki-ns1")

# 검색 결과를 기반으로 context를 생성합니다.
context = ""
for result in results:
    context += result.page_content

# 프롬프트 템플릿을 사용하여 메시지를 포맷합니다.
prompt = chat_template.format_messages(country=country, context=context)
print(prompt)

# LLM을 사용하여 프롬프트에 대한 응답을 생성합니다.
response = llm.invoke(prompt)
print(response.content)